In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 背景和目标

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
display(values)
display(pd.unique(values))
display(pd.value_counts(values))

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

array(['apple', 'orange'], dtype=object)

apple     6
orange    2
dtype: int64

In [3]:
# 更有效的存储方式
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
display(values)
display(dim)

dim.take(values)  # 使用take实现类似于字典翻译的效果

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

0     apple
1    orange
dtype: object

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### pandas中的Categorical类型

In [4]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                   columns=['basket_id', 'fruit', 'count', 'weight'])
display(df)

fruit_cat = df['fruit'].astype('category')  # fruit_cat是pandas.Categorical类型
fruit_cat

,basket_id,fruit,count,weight
0,0,apple,8,2.159908
1,1,orange,5,2.096293
2,2,apple,11,3.398954
3,3,apple,8,3.159545
4,4,apple,5,2.436073
5,5,orange,6,1.585501
6,6,apple,7,0.802782
7,7,apple,12,2.353512


0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [5]:
c = fruit_cat.values

display(type(c))
display(c.categories)
display(c.codes)

pandas.core.arrays.categorical.Categorical

Index(['apple', 'orange'], dtype='object')

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [6]:
df['fruit'] = df['fruit'].astype('category')  # 将DataFrame一列的结果转换为Categorical对象
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [7]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [8]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [9]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)  # 除非显示指定，否则分类数组默认是没有顺序的
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [10]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### 使用Categorical对象进行计算

In [11]:
np.random.seed(12345)
draws = np.random.randn(1000)
display(draws[:5])

bins = pd.qcut(draws, 4)
display(bins)

bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
display(bins)
display(bins.codes[:10])

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [12]:
bins = pd.Series(bins, name='quartile')
results = pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index()
display(results)
display(results['quartile'])

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### 使用分类数据获取更高的性能

如果你是在一个特定数据集上做大量分析，将其转换为分类可以极大地提高效率。DataFrame列的分类使用的内存通常少的多。
GroupBy使用分类操作明显更快，是因为底层的算法使用整数编码数组，而不是字符串数组。

In [13]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

categories = labels.astype('category')

# 内存比较
display(labels.memory_usage())
display(categories.memory_usage())

# 转换的一次性开销
%time _ = labels.astype('category')

80000128

10000332

CPU times: user 890 ms, sys: 20.1 ms, total: 910 ms
Wall time: 908 ms


In [14]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
display(cat_s)

display(cat_s.cat.codes)  # 特殊属性cat提供了对分类数据的访问
display(cat_s.cat.categories)

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

Index(['a', 'b', 'c', 'd'], dtype='object')

In [15]:
actual_categories = ['a', 'b', 'c', 'd', 'e']  # 假设这个数据的实际分类集超出了数据中的四个值。可以使用set_categories方法改变它们：
cat_s2 = cat_s.cat.set_categories(actual_categories)
display(cat_s2)

display(cat_s.value_counts())
display(cat_s2.value_counts())

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

a    2
b    2
c    2
d    2
dtype: int64

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [16]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
display(cat_s3)

cat_s3.cat.remove_unused_categories()  # 移除“未观察”到的项

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

#### 创建用于建模的虚拟变量

In [17]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1
